In [230]:
###########################################################################################
# Description: Program to generate text using various models                                                    
# Author: Bhaskar Mahna
# Revision History:
############################################################################################
import numpy as np
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU

from keras.models import Sequential


In [231]:
# Read lines from an example source file.
with open("travel_blogs", 'rb') as file1:
    lines = []
    for line in file1:
        line = line.strip().lower().decode("ascii", "ignore")
        print(line)
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

if you are looking forward to a wide variety of landscapes and multiple choices of anchoring on a sailing holiday, then sail along the beautiful coastline of mallorca. the typical setting amidst the mallorcas coastline means admiring the shallow turquoise waters against the theatrical mountains and dive into the sea whenever one wants. you can halt at those secluded beaches dotted with idyllic coves and dunes. you would just want the time to stand still and enjoy the glorious weather, the fantastic sea, and the excellent sailing trip.
the spanish islands of mallorca boast of a charming country lifestyle and are a realm of tradition and history. if you are toying with the idea of planning a sailing holiday with your family and friends around mallorca, well, you are in for a perfect holiday trip and memories of a lifetime. choose reputed sites like www.sailingholidays.com for that ideal vacation.
mallorca, also known as majorca, is the largest island in the balearic islands and is locate

In [232]:
# For convenience, choose a fixed sequence length of 10 characters.
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []
print(len(text))
# Convert the data into a series of different SEQLEN-length subsequences.
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

print(len(input_chars))
print(len(label_chars))

# Create a character index and reverse mapping to go between a numerical
# ID and a specific character. The numerical ID will correspond to a column
# number when using a one-hot encoded representation of character inputs.
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

6179
6169
6169


In [233]:

# Compute the one-hot encoding of the input sequences X and the next
# character (the label) y
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
#y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
    

In [242]:

#Setup of Hyper parameter 
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 100
NUM_PREDS_PER_EPOCH = 300


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.
model = Sequential()
model.add(
        SimpleRNN(  # Note you can vary this with GRU, LSTM or SimpleRNN to try alternatives.
        HIDDEN_SIZE,
        return_sequences=True,
        go_backwards=True, 
        dropout=0.2,
        recurrent_dropout=0.2,    
        input_shape=(SEQLEN, nb_chars)
        #input_shape=(None, nb_chars),
        #unroll=True
    )
)
#keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
#kernel_initializer='glorot_uniform', 
#recurrent_initializer='orthogonal', bias_initializer='zeros', 
#unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, 
#bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, 
#bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, 
#return_state=False, go_backwards=False, stateful=False, unroll=False)

model.add(SimpleRNN(HIDDEN_SIZE))
model.add(Dense(nb_chars))
model.add(Activation("relu"))
#model.add(Activation("tanh"))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 10, 128)           81408     
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_28 (Dense)             (None, 30)                3870      
_________________________________________________________________
activation_41 (Activation)   (None, 30)                0         
_________________________________________________________________
activation_42 (Activation)   (None, 30)                0         
Total params: 216,862
Trainable params: 216,862
Non-trainable params: 0
_________________________________________________________________


In [243]:
import matplotlib.pyplot as plt

# Execute a series of training and demonstration iterations.
for iteration in range(NUM_ITERATIONS):

    # For each iteration, run the model fitting procedure for a number of epochs.
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example from input text sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # Following loop is for getting prediction for no. of characters predic using the current version of the trained
    # model 
    # construct a one-hot encoding of the randomly generated test input and append a prediction
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # Here is the one-hot encoding of seed text
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # Make a prediction with the current model.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # Print the prediction appended to the test example.
        print(y_pred, end="")

        # Increment the test example to contain the prediction as if it
        # were the correct next letter.
        test_chars = test_chars[1:] + y_pred


print()

Iteration #: 0
Epoch 1/100
6169/6169 [==============================] - 12s 2ms/step - loss: 3.0465 - acc: 0.1675
Epoch 2/100
6169/6169 [==============================] - 4s 648us/step - loss: 2.9671 - acc: 0.1942
Epoch 3/100
6169/6169 [==============================] - 4s 654us/step - loss: 2.9250 - acc: 0.2065
Epoch 4/100
6169/6169 [==============================] - 4s 708us/step - loss: 2.8966 - acc: 0.2182
Epoch 5/100
6169/6169 [==============================] - 4s 639us/step - loss: 2.8613 - acc: 0.2253 1s - loss: 2.
Epoch 6/100
6169/6169 [==============================] - 4s 645us/step - loss: 2.7879 - acc: 0.2342
Epoch 7/100
6169/6169 [==============================] - 4s 643us/step - loss: 2.7114 - acc: 0.2372
Epoch 8/100
6169/6169 [==============================] - 4s 655us/step - loss: 2.6839 - acc: 0.2385
Epoch 9/100
6169/6169 [==============================] - 4s 613us/step - loss: 2.6548 - acc: 0.2534
Epoch 10/100
6169/6169 [==============================] - 4s 631us/step 

6169/6169 [==============================] - 4s 696us/step - loss: 1.9576 - acc: 0.4357
Epoch 81/100
6169/6169 [==============================] - 4s 661us/step - loss: 1.9468 - acc: 0.4406
Epoch 82/100
6169/6169 [==============================] - 4s 651us/step - loss: 1.9465 - acc: 0.4416
Epoch 83/100
6169/6169 [==============================] - 4s 648us/step - loss: 1.9465 - acc: 0.4396
Epoch 84/100
6169/6169 [==============================] - 4s 701us/step - loss: 1.9242 - acc: 0.4450
Epoch 85/100
6169/6169 [==============================] - 4s 675us/step - loss: 1.9164 - acc: 0.4497
Epoch 86/100
6169/6169 [==============================] - 4s 669us/step - loss: 1.9198 - acc: 0.4472 2s 
Epoch 87/100
6169/6169 [==============================] - 4s 687us/step - loss: 1.9034 - acc: 0.4596
Epoch 88/100
6169/6169 [==============================] - 5s 730us/step - loss: 1.8978 - acc: 0.4505
Epoch 89/100
6169/6169 [==============================] - 4s 658us/step - loss: 1.9040 - acc: 0.4537

Epoch 56/100
6169/6169 [==============================] - 4s 722us/step - loss: 1.5483 - acc: 0.5529
Epoch 57/100
6169/6169 [==============================] - 4s 697us/step - loss: 1.5567 - acc: 0.5469
Epoch 58/100
6169/6169 [==============================] - 5s 764us/step - loss: 1.5556 - acc: 0.5547
Epoch 59/100
6169/6169 [==============================] - 4s 703us/step - loss: 1.5430 - acc: 0.5524
Epoch 60/100
6169/6169 [==============================] - 4s 688us/step - loss: 1.5328 - acc: 0.5610
Epoch 61/100
6169/6169 [==============================] - 4s 699us/step - loss: 1.5325 - acc: 0.5520
Epoch 62/100
6169/6169 [==============================] - 5s 768us/step - loss: 1.5202 - acc: 0.5594
Epoch 63/100
6169/6169 [==============================] - 4s 715us/step - loss: 1.5089 - acc: 0.5630
Epoch 64/100
6169/6169 [==============================] - 5s 759us/step - loss: 1.5218 - acc: 0.5558
Epoch 65/100
6169/6169 [==============================] - 4s 716us/step - loss: 1.5068 - ac

6169/6169 [==============================] - 4s 669us/step - loss: 0.8981 - acc: 0.7453
Epoch 6/100
6169/6169 [==============================] - 4s 711us/step - loss: 0.8981 - acc: 0.7513
Epoch 7/100
6169/6169 [==============================] - 4s 711us/step - loss: 0.8884 - acc: 0.7453
Epoch 8/100
6169/6169 [==============================] - 4s 672us/step - loss: 0.8753 - acc: 0.7518
Epoch 9/100
6169/6169 [==============================] - 4s 687us/step - loss: 0.8791 - acc: 0.7523
Epoch 10/100
6169/6169 [==============================] - 4s 699us/step - loss: 0.8598 - acc: 0.7583
Epoch 11/100
6169/6169 [==============================] - 4s 679us/step - loss: 0.8927 - acc: 0.7483
Epoch 12/100
6169/6169 [==============================] - 4s 676us/step - loss: 0.8752 - acc: 0.7504
Epoch 13/100
6169/6169 [==============================] - 4s 681us/step - loss: 0.8733 - acc: 0.7544
Epoch 14/100
6169/6169 [==============================] - 4s 698us/step - loss: 0.8592 - acc: 0.7590
Epoch 1

6169/6169 [==============================] - 5s 730us/step - loss: 0.5288 - acc: 0.8583
Epoch 58/100
6169/6169 [==============================] - 4s 713us/step - loss: 0.5426 - acc: 0.8499 1s - loss:
Epoch 59/100
6169/6169 [==============================] - 4s 712us/step - loss: 0.5565 - acc: 0.8491
Epoch 60/100
6169/6169 [==============================] - 5s 744us/step - loss: 0.5467 - acc: 0.8481 1s - loss: 
Epoch 61/100
6169/6169 [==============================] - 4s 716us/step - loss: 0.5511 - acc: 0.8520 0s - loss: 0.5474 - acc: 0
Epoch 62/100
6169/6169 [==============================] - 4s 712us/step - loss: 0.5221 - acc: 0.8559
Epoch 63/100
6169/6169 [==============================] - 4s 729us/step - loss: 0.5364 - acc: 0.8473
Epoch 64/100
6169/6169 [==============================] - 4s 728us/step - loss: 0.5225 - acc: 0.8591
Epoch 65/100
6169/6169 [==============================] - 4s 716us/step - loss: 0.5321 - acc: 0.8505 1s - loss: 0.52 - ETA: 0s - loss: 0.5306 - acc: 
Epoch

6169/6169 [==============================] - 5s 842us/step - loss: 0.3925 - acc: 0.8919
Epoch 4/100
6169/6169 [==============================] - 5s 779us/step - loss: 0.4005 - acc: 0.8886 1s - loss: 0.4048 - 
Epoch 5/100
6169/6169 [==============================] - 5s 750us/step - loss: 0.3936 - acc: 0.8890
Epoch 6/100
6169/6169 [==============================] - 5s 757us/step - loss: 0.3988 - acc: 0.8894
Epoch 7/100
6169/6169 [==============================] - 5s 745us/step - loss: 0.4011 - acc: 0.8901 2s
Epoch 8/100
6169/6169 [==============================] - 5s 735us/step - loss: 0.4061 - acc: 0.8864
Epoch 9/100
6169/6169 [==============================] - 5s 742us/step - loss: 0.4075 - acc: 0.8864
Epoch 10/100
6169/6169 [==============================] - 5s 767us/step - loss: 0.4087 - acc: 0.8864 1s - loss: 0.4224 - 
Epoch 11/100
6169/6169 [==============================] - 5s 749us/step - loss: 0.3962 - acc: 0.8930 0s - loss: 0.3949 - ac
Epoch 12/100
6169/6169 [==================

6169/6169 [==============================] - 5s 738us/step - loss: 0.3353 - acc: 0.9047
Epoch 58/100
6169/6169 [==============================] - 5s 744us/step - loss: 0.3328 - acc: 0.9052
Epoch 59/100
6169/6169 [==============================] - 5s 765us/step - loss: 0.3290 - acc: 0.9074 4s - loss: 0.3532 - acc: 0 
Epoch 60/100
6169/6169 [==============================] - 5s 738us/step - loss: 0.3352 - acc: 0.9060
Epoch 61/100
6169/6169 [==============================] - 5s 751us/step - loss: 0.3320 - acc: 0.9061
Epoch 62/100
6169/6169 [==============================] - 5s 757us/step - loss: 0.3350 - acc: 0.9066
Epoch 63/100
6169/6169 [==============================] - 5s 751us/step - loss: 0.3283 - acc: 0.9076
Epoch 64/100
6169/6169 [==============================] - 5s 739us/step - loss: 0.3446 - acc: 0.9023
Epoch 65/100
6169/6169 [==============================] - 5s 740us/step - loss: 0.3347 - acc: 0.9071
Epoch 66/100
6169/6169 [==============================] - 5s 766us/step - lo

6169/6169 [==============================] - 5s 780us/step - loss: 0.3124 - acc: 0.9113
Epoch 8/100
6169/6169 [==============================] - 5s 756us/step - loss: 0.3102 - acc: 0.9099
Epoch 9/100
6169/6169 [==============================] - 5s 745us/step - loss: 0.3062 - acc: 0.9107  - ETA: 1s - loss: 0.3005 - acc: 0.9 - ETA: 1s - loss: 0.3010 -
Epoch 10/100
6169/6169 [==============================] - 5s 773us/step - loss: 0.3024 - acc: 0.9120
Epoch 11/100
6169/6169 [==============================] - 5s 760us/step - loss: 0.2969 - acc: 0.9126
Epoch 12/100
6169/6169 [==============================] - 5s 747us/step - loss: 0.2980 - acc: 0.9157
Epoch 13/100
6169/6169 [==============================] - 5s 756us/step - loss: 0.3016 - acc: 0.9136 0s - loss: 0.3026 - acc: 0.9
Epoch 14/100
6169/6169 [==============================] - 5s 775us/step - loss: 0.3059 - acc: 0.9105
Epoch 15/100
6169/6169 [==============================] - 5s 746us/step - loss: 0.2961 - acc: 0.9141 0s - loss: 0.

In [28]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
